<a href="https://colab.research.google.com/github/swati1033/CodTech--Task-1/blob/main/Hybrid_Approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
data = pd.read_csv("fashion_products.csv")
data.head(10)

,User ID,Product ID,Product Name,Brand,Category,Price,Rating,Color,Size
0,19,1,Dress,Adidas,Men's Fashion,40,1.043159,Black,XL
1,97,2,Shoes,H&M,Women's Fashion,82,4.026416,Black,L
2,25,3,Dress,Adidas,Women's Fashion,44,3.337938,Yellow,XL
3,57,4,Shoes,Zara,Men's Fashion,23,1.049523,White,S
4,79,5,T-shirt,Adidas,Men's Fashion,79,4.302773,Black,M
5,98,6,Dress,Adidas,Men's Fashion,47,1.379566,Yellow,L
6,16,7,Jeans,Gucci,Men's Fashion,37,1.356750,White,XL
7,63,8,Sweater,Zara,Kids' Fashion,64,4.360303,Blue,XL
8,96,9,Sweater,H&M,Men's Fashion,53,4.466182,Green,XL
9,36,10,T-shirt,Zara,Kids' Fashion,55,4.093234,White,XL


In [ ]:
data.isnull().sum()

,0
User ID,0
Product ID,0
Product Name,0
Brand,0
Category,0
Price,0
Rating,0
Color,0
Size,0


In [ ]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357271 sha256=ca14c2d8982b42f2f2ef7ab65398044290289aab8d2f819a29851500f12bbc76
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [ ]:
import pandas as pd
from surprise import Reader, Dataset, SVD
from surprise.model_selection import train_test_split
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer

# Load data
data_df = pd.read_csv('/content/fashion_products.csv')

# Step 1: Prepare the data
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(data_df[['User ID', 'Product ID', 'Rating']], reader)

# Step 2: Split the data (set random_state for reproducibility)
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Step 3: Train the model using SVD (set random_state for reproducibility)
svd = SVD(random_state=42)
svd.fit(trainset)

# Content-Based Filtering (using TF-IDF for content similarity)
content_df = data_df[['Product ID', 'Product Name', 'Brand', 'Category', 'Color', 'Size']]
content_df['Content'] = content_df.apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)

tfidf_vectorizer = TfidfVectorizer()
content_matrix = tfidf_vectorizer.fit_transform(content_df['Content'])
content_similarity = linear_kernel(content_matrix, content_matrix)

# Function to get content-based recommendations
def get_content_based_recommendations(product_id, top_n):
    index = content_df[content_df['Product ID'] == product_id].index[0]
    similarity_scores = content_similarity[index]
    similar_indices = similarity_scores.argsort()[::-1][1:top_n + 1]
    recommended_products = content_df.loc[similar_indices, 'Product ID'].values
    return recommended_products

# Function to get predicted rating for a user-product pair
def get_predicted_rating(user_id, product_id):
    prediction = svd.predict(user_id, product_id)
    return prediction.est

# Hybrid Recommendations with Predicted Ratings
def get_hybrid_recommendations_with_ratings(user_id, product_id, top_n):
    recommended_products = get_content_based_recommendations(product_id, top_n)

    # Predict ratings for the recommended products
    recommendations_with_ratings = []
    for product in recommended_products:
        predicted_rating = get_predicted_rating(user_id, product)
        recommendations_with_ratings.append((product, predicted_rating))

    return recommendations_with_ratings

# Example usage
user_id = 79
product_id = 5
top_n = 10
recommendations_with_ratings = get_hybrid_recommendations_with_ratings(user_id, product_id, top_n)

# Display results
print(f"Hybrid Recommendations for User {user_id} based on Product {product_id}:")
for i, (product_id, rating) in enumerate(recommendations_with_ratings):
    print(f"{i + 1}. Product ID: {product_id}, Predicted Rating: {rating:.2f}")


Hybrid Recommendations for User 79 based on Product 5:
1. Product ID: 1, Predicted Rating: 2.97
2. Product ID: 221, Predicted Rating: 2.85
3. Product ID: 989, Predicted Rating: 3.21
4. Product ID: 331, Predicted Rating: 2.92
5. Product ID: 570, Predicted Rating: 2.71
6. Product ID: 817, Predicted Rating: 2.78
7. Product ID: 890, Predicted Rating: 2.83
8. Product ID: 6, Predicted Rating: 2.75
9. Product ID: 423, Predicted Rating: 3.05
10. Product ID: 720, Predicted Rating: 2.92


<ipython-input-42-98f17510cb9c>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content_df['Content'] = content_df.apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)


In [ ]:
import numpy as np

# Actual ratings of user for recommended products (these would be from your dataset)
# Example actual ratings: replace with the real data
actual_ratings = [1.0, 2.8, 4.7, 3.2, 1.4, 1.1, 1.0, 1.3, 4.5, 2.5]

# Predicted ratings from the recommendation system (this should be the predicted ratings for the top-n recommendations)
# Example predicted ratings: replace with the real predicted ratings
predicted_ratings = [2.9, 2.8,3.2,2.9,2.7,2.7,2.8,2.7,3.0,2.9]

# Calculate RMSE
rmse = np.sqrt(np.mean((np.array(actual_ratings) - np.array(predicted_ratings))**2))
print(f"RMSE: {rmse}")


RMSE: 1.334541119636259


In [ ]:
def calculate_accuracy_from_rmse(rmse, min_rating=1, max_rating=5):
    accuracy = 1 - (rmse / (max_rating - min_rating))
    return accuracy

# Example usage:
rmse = 1.31  # RMSE value you calculated earlier
accuracy = calculate_accuracy_from_rmse(rmse)

print(f"Accuracy based on RMSE ({rmse}): {accuracy:.2f}")


Accuracy based on RMSE (1.31): 0.67
